<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/tensorflow/tf_low_level_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Level TensorFlow, Part II: Training

* https://www.tensorflow.org/guide/low_level_intro

In [1]:
# import and check version
import tensorflow as tf
# tf can be really verbose
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.12.0


In [2]:
# a small sanity check, does tf seem to work ok? 
sess = tf.Session()
hello = tf.constant('Hello TF!')
print(sess.run(hello))
sess.close()

b'Hello TF!'


## First define a computational graph composed of operations and tensors

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


## Then use a session to execute the graph

In [4]:
# sessions need to be closed in order not to leak ressources, this makes sure close is called in any case
with tf.Session() as sess:
  print(sess.run(total))

7.0


## Graphs can be executed on CPU, GPU, and even TPU

In [5]:
# let's see what compute devices we have available, hopefully a GPU 
# if you do not see it, switch on under Runtime->Change runtime type
with tf.Session() as sess:
  devices = sess.list_devices()
  for d in devices:
      print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:XLA_CPU:0
/job:localhost/replica:0/task:0/device:XLA_GPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
with tf.device("/device:XLA_CPU:0"):
  with tf.Session() as sess:
    print(sess.run(total))

7.0


## Feeding data to a graph

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [8]:
with tf.Session() as sess:
  try:
    print(sess.run(z))
  except tf.errors.InvalidArgumentError as iae:
     print(iae.message)

You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[node Placeholder (defined at <ipython-input-7-3b59dde2b9f0>:1)  = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[{{node add_1/_1}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7_add_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [9]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3.0, y: 4.5}))

7.5


## Reading in data sets

In [10]:
r = tf.random_normal([10, 2])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
  sess.run(iterator.initializer)
  while True:
    try:
      data = sess.run(next_row)
      print(data)
      print(sess.run(z, feed_dict={x: data[0], y: data[1]}))
    except tf.errors.OutOfRangeError:
      break

[-0.59269613 -1.0008177 ]
-1.5935137
[ 0.5592084  -0.98474556]
-0.42553717
[ 1.2175804  -0.80723673]
0.4103437
[0.28345087 0.661987  ]
0.9454379
[-1.2553709   0.46523964]
-0.7901312
[-0.2941359  1.6356422]
1.3415062
[-0.54164934 -1.6740425 ]
-2.2156918
[-0.32486457 -0.5762184 ]
-0.901083
[0.98022515 0.79792726]
1.7781525
[ 1.6103048  -0.54175496]
1.0685499


## Layers

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(inputs=x, units=1)

with tf.Session() as sess:
  try:
    print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))
  except tf.errors.FailedPreconditionError as fpe:
    print(fpe.message)

Attempting to use uninitialized value dense/kernel
	 [[node dense/kernel/read (defined at <ipython-input-11-eb68a0e5f9f4>:2)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense/kernel)]]
	 [[{{node dense/BiasAdd/_3}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_12_dense/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [12]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[0.8728956]
 [3.037486 ]]


In [13]:
y = tf.layers.dense(inputs=x, units=2, activation=tf.nn.tanh)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[0.75823957 0.99993545]
 [0.99061966 1.        ]]


## Feature columns

transform a diverse range of raw data into formats input layers can accept

* https://www.tensorflow.org/guide/feature_columns
* https://www.tensorflow.org/api_docs/python/tf/feature_column/input_layer

In [0]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}

# numeric values are simple
sales_column = tf.feature_column.numeric_column('sales')
columns = {
   sales_column
}

inputs = tf.feature_column.input_layer(features, columns)

In [15]:
# categories are harders, as NNs only accept dense numeric values

categorical_department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])

columns = {
  sales_column,
  categorical_department_column
}

# we can decide if we want the category to be encoded as embedding or multi-hot 
try:
  inputs = tf.feature_column.input_layer(features, columns)
except ValueError as ve:
  print(ve)

Items of feature_columns must be a _DenseColumn. You can wrap a categorical column with an embedding_column or indicator_column. Given: _VocabularyListCategoricalColumn(key='department', vocabulary_list=('sports', 'gardening'), dtype=tf.string, default_value=-1, num_oov_buckets=0)


In [0]:
multi_hot_department_column = tf.feature_column.indicator_column(categorical_department_column)

In [0]:
columns = {
  sales_column,
  multi_hot_department_column
}

inputs = tf.feature_column.input_layer(features, columns)

In [21]:
# feature columns also need initialization
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  # first two are departments last entry is just sales as is
  print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


In [22]:
# multi (one in our case) hot encoding of departments
columns = {
  multi_hot_department_column
}

inputs = tf.feature_column.input_layer(features, columns)
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  print(sess.run(inputs))


[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [33]:
# alternative, embedding in three dimensions
embedding_department_column = tf.feature_column.embedding_column(categorical_department_column, dimension=3)
columns = {
  embedding_department_column
}

inputs = tf.feature_column.input_layer(features, columns)
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
with tf.Session() as sess:
  sess.run((var_init, table_init))
  print(sess.run(inputs))

[[-0.14460449  0.24125229 -0.10900439]
 [-0.14460449  0.24125229 -0.10900439]
 [-0.30600104  0.80903935 -0.7531071 ]
 [-0.30600104  0.80903935 -0.7531071 ]]
